In [1]:
from wordnet import *
from Udep2Mono.binarization import BinaryDependencyTree
from Udep2Mono import polarization
from Udep2Mono.util import btreeToList
from copy import deepcopy

2020-12-21 22:59:29 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | gum       |
| pos       | gum       |
| lemma     | gum       |
| depparse  | gum       |
| ner       | ontonotes |

2020-12-21 22:59:29 INFO: Use device: gpu
2020-12-21 22:59:29 INFO: Loading: tokenize
2020-12-21 22:59:32 INFO: Loading: pos
2020-12-21 22:59:33 INFO: Loading: lemma
2020-12-21 22:59:33 INFO: Loading: depparse
2020-12-21 22:59:35 INFO: Loading: ner
2020-12-21 22:59:36 INFO: Done loading processors!


In [2]:
from pattern.en import pluralize, singularize

class LexicalGenerator:
    def __init__(self):
        self.deptree = None
        self.length = 0
        self.tree_factory = TreeFactory()
        self.treeLog = []
        self.polarLog = []
        self.lexial_generation = {
             "acl": self.generate_acl_relcl,
             "acl:relcl": self.generate_acl_relcl,
             "advcl": self.generate_acl_relcl,
             "cc:preconj": self.generate_det,
             "det": self.generate_det,
             "det:predet": self.generate_det,
        }
        '''"advmod": self.generate_advmod,
            "advmod:count": self.generate_advmod,
            "amod": self.generate_amod,
            "appos": self.generate_inherite,
            "aux": self.generate_aux,
            "aux:pass": self.generate_aux,
            "case": self.generate_case,
            "cc": self.generate_cc,
           
            "ccomp": self.generate_ccomp,
            "compound": self.generate_inherite,
            "compound:prt": self.generate_inherite,
            "conj": self.generate_inherite,
            "cop": self.generate_inherite,
            "csubj": self.generate_nsubj,
            "csubj:pass": self.generate_nsubj,
            "dep": self.generate_dep,
            
            "discourse": self.generate_discourse,
            "expl": self.generate_expl,
            "fixed": self.generate_inherite,
            "flat": self.generate_inherite,
            "goeswith": self.generate_inherite,
            "iobj": self.generate_inherite,
            "mark": self.generate_inherite,
            "nmod": self.generate_nmod,
            "nmod:npmod": self.generate_nmod,
            "nmod:tmod": self.generate_nmod,
            "nmod:poss": self.generate_nmod_poss,
            "nsubj": self.generate_nsubj,
            "nsubj:pass": self.generate_nsubj,
            "nummod": self.generate_nummod,
            "obj": self.generate_obj,
            "obl": self.generate_obj,
            "obl:npmod": self.generate_oblnpmod,
            "obl:tmod": self.generate_inherite,
            "parataxis": self.generate_inherite,
            "xcomp": self.generate_obj,'''

    def deptree_generate(self, length, tree):
        self.deptree = tree
        self.length = length
        self.generate(self.deptree)

    def generate(self, tree):
        if tree.val in self.lexial_generation.keys():
            self.lexial_generation[tree.val](tree)

    def save_tree(self, tree=None):
        if tree is not None:
            generated, _, _, _ = btreeToList(tree, self.length, {}, 0)
        else:
            generated, _, _, _ = btreeToList(self.deptree, self.length, {}, 0)
        generated = '[%s]' % ', '.join(map(str, generated)).replace("'", "")
        generated = generated.replace(",", "")
        print("New tree: ", generated)

        if tree is not None:
            return deepcopy(self.deptree)
        else:
            return deepcopy(self.deptree)

    def rollback(self, tree, backup):
        tree.val = backup.val
        tree.left = deepcopy(backup.left)
        tree.right = deepcopy(backup.right)
        tree.mark = backup.mark
        tree.npos = backup.npos
        tree.id = backup.id

    def generate_amod(self, tree):  
        "amod + Noun => Noun"
        left = tree.left 
        right = tree.right
        backup = deepcopy(tree)
        hyper = [] 
        hypo = [] 
        syn =[] 
        
        if right.npos != None:
            hyper, hypo, syn, ant = get_word_sets(
                singularize(right.val), right.npos.lower())

            if right.mark == "+":                
                for word in hyper:
                    tree.right.val = word
                    self.treeLog.append(self.save_tree())
                self.rollback(tree, backup)

            if right.mark == "-":
                for word in hypo:
                    tree.right.val = word
                    self.treeLog.append(self.save_tree())
                self.rollback(tree, backup)
        else:
            self.generate(left)
                

    def generate_acl_relcl(self, tree):
        left = tree.left
        right = tree.right
        backup = deepcopy(tree)
        hyper = [] 
        hypo = [] 
        syn =[] 

        if right.npos != None:
            hyper, hypo, syn, ant = get_word_sets(
                singularize(right.val), right.npos.lower())

            if right.mark == "+":                
                for word in hyper:
                    tree.right.val = word
                    self.treeLog.append(self.save_tree())
                self.rollback(tree, backup)

            if right.mark == "-":
                for word in hypo:
                    tree.right.val = word
                    self.treeLog.append(self.save_tree())
                self.rollback(tree, backup)
        else:
            self.generate(left)

    def generate_det(self, tree):
        left = tree.left
        right = tree.right
        backup = deepcopy(tree)
        #hyper, hypo, syn, ant = get_word_sets(right.val, right.npos.lower())
        kb = quantifier.find({"word": left.val})[0]
        self.generate(right)

        if left.mark == "+":
            for word in kb["<"]:
                tree.left.val = word
                self.treeLog.append(self.save_tree())
            self.rollback(tree, backup)
        
        if left.mark == "-":
            for word in kb[">"]:
                tree.left.val = word
                self.treeLog.append(self.save_tree())
            self.rollback(tree, backup)            

In [ ]:
sentences = ["All flowers are beautiful"]
annotations, _ = polarization.run_polarize_pipeline(
    sentences, verbose=2, parser="stanza")

lexicalGenerator = LexicalGenerator()
for annotation in annotations:
    annotated, original, polarized, postags, polarized_tree = annotation
    print(annotated)
    lexicalGenerator.deptree_generate(len(original), polarized_tree)